In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Layout, widgets, Dropdown, VBox, HBox, interactive
from IPython.display import display, clear_output
import scipy.signal
import scipy.special
import time
from scipy.signal import upfirdn


# Root Raised Cosine (RRC) filter function
def rootRaisedCosine1(nsamp, roll_off, delay):
    # Time vector for filter based on delay and samples per symbol
    t = np.arange(-delay, delay + 1 / nsamp, 1 / nsamp)
    h = np.zeros(len(t))  # Initialize filter coefficients

    # Loop to calculate filter coefficients
    for i in range(len(t)):
        if t[i] == 0.0:  # Special case for t = 0
            h[i] = 1.0 - roll_off + 4 * roll_off / np.pi
        elif roll_off != 0 and t[i] == 1 / (4 * roll_off):  # Special case for specific t
            h[i] = roll_off / np.sqrt(2) * (
                (1 + 2 / np.pi) * np.sin(np.pi / (4 * roll_off)) + 
                (1 - 2 / np.pi) * np.cos(np.pi / (4 * roll_off))
            )
        elif roll_off != 0 and t[i] == -1 / (4 * roll_off):  # Symmetric case for t
            h[i] = roll_off / np.sqrt(2) * (
                (1 + 2 / np.pi) * np.sin(np.pi / (4 * roll_off)) + 
                (1 - 2 / np.pi) * np.cos(np.pi / (4 * roll_off))
            )
        else:  # General case
            h[i] = (np.sin(np.pi * t[i] * (1 - roll_off)) + 
                    4 * roll_off * t[i] * np.cos(np.pi * t[i] * (1 + roll_off))) / (
                    np.pi * t[i] * (1 - (4 * roll_off * t[i]) ** 2))
    
    return h  # Return filter coefficients

# Function to compute theoretical BER for M-PSK
def compute_ber_psk(EbNo_dB, M1):
    EbNo_linear = 10**(EbNo_dB / 10)  # Convert dB to linear scale
    if M1 == 2:  # BPSK case
        return 0.5 * scipy.special.erfc(np.sqrt(EbNo_linear))
    else:  # M-PSK case
        k = np.log2(M1)  # Bits per symbol
        return (1 / 4 * k) * scipy.special.erfc(np.sqrt(EbNo_linear * k) * np.sin(np.pi / M1))

# Function to simulate BER for M-PSK
def ber_psk_simulation(EbNo_dB, M1):
    Nsymb = 30000  # Number of symbols
    nsamp = 16  # Samples per symbol
    fc = 4  # Carrier frequency
    rolloff = 0.25  # Roll-off factor
    delay = 10  # Filter delay

    # Calculate SNR in dB
    SNR_dB = EbNo_dB - 10 * np.log10(nsamp / np.log2(M1))

    # Generate RRC filter
    shaping_filter = rootRaisedCosine1(nsamp, rolloff, delay)
    
    # Generate random symbols
    bits1 = np.random.randint(0, M1, Nsymb)
    
    # Map bits to PSK symbols
    symbols = np.exp(1j * (2 * np.pi * bits1 / M1))
    
    # Upsample and filter transmitted signal
    ytx1 = upfirdn([1], symbols, nsamp)
    ytx1 = np.convolve(ytx1, shaping_filter, mode='same')

    # Modulate with carrier frequency
    m1 = np.arange(len(ytx1))
    s1 = np.real(ytx1 * np.exp(1j * 2 * np.pi * fc * m1 / nsamp))

    # Calculate signal power and noise power
    Ps = np.mean(np.abs(s1)**2)
    SNR_linear = 10**(SNR_dB / 10)
    Pn = Ps / SNR_linear
    noise = np.sqrt(Pn / 4) * (np.random.randn(len(s1)) + 1j * np.random.randn(len(s1)))

    # Add noise to the signal
    snoisy = s1 + noise

    # Demodulate received signal
    yrx1 = snoisy * np.exp(-1j * 2 * np.pi * fc * m1 / nsamp)
    yrx1 = np.convolve(yrx1, shaping_filter, mode='same')
    yrx1 = yrx1[::nsamp]

    # Detect symbols by mapping angles back to symbol indices
    detected_bits1 = np.angle(yrx1) * M1 / (2 * np.pi)
    detected_bits1 = np.round(detected_bits1) % M1

    # Calculate BER by comparing transmitted and received symbols
    bit_errors1 = np.sum(bits1 != detected_bits1)
    ber1 = bit_errors1 / len(bits1)

    return ber1  # Return the BER

# Function to plot BER curve for PSK modulation
def plot_ber_psk(M):
    ber_exp = []  # List for experimental BER values
    ber_th = []  # List for theoretical BER values

    # Loop through Eb/N0 values in dB
    for i in range(1, 18):
        ber_exp.append(ber_psk_simulation(i, M))  # Simulated BER
        ber_th.append(compute_ber_psk(i, M))  # Theoretical BER

    # Plot the results
    plt.figure(figsize=(10, 8))

    # Plot theoretical BER curve
    plt.semilogy(range(1, 18), ber_th)

    # Plot experimental BER points
    plt.semilogy(range(1, 18), ber_exp, 'o')

    # Add labels, title, and grid
    plt.legend(['Theoretical', 'Simulation'])
    plt.xlabel('Eb/N0 (dB)')
    plt.ylabel('Bit Error Probability')
    plt.title(f'BER curve for {M}-PSK')
    plt.grid(which='both')
    plt.show()  # Display the plot


# Define PSK options
psk_options = {'BPSK': 2, 'QPSK': 4, '8-PSK': 8}
psk_selector = widgets.Dropdown(options=psk_options, value=4, description='PSK Type:')

# Create interactive widget
interactive_plot = interactive(plot_ber_psk, M=psk_selector)

input_widgets = VBox([psk_selector], layout=Layout(width='auto'))
plot_output = interactive_plot.children[-1]  # The output plot

# Create a VBox that includes both the input widgets and the loading animation
inputs = HBox([input_widgets], layout=Layout(align_items='center'))

# Create an HBox to hold everything in a horizontal layout
ui = VBox([inputs, plot_output])

# Display the UI components
clear_output(wait=True)  # Clear the previous output
display(ui)
